---
title: Detecting abnormal attendance patterns from daily data
subtitle: A technical presentation - 14th March 2025
title-slide-attributes:
  data-background-position: top
  data-background-image: images/sanctuary-buildings.jpg
  data-background-size: cover
  data-background-opacity: "0.1"
author: "Finn Trinci"
institute: Department for Education
format:
  revealjs:
    theme: [dark, dfe-quarto-slides.scss]
    width: 120%
    height: 120%
    logo: images/DfE_logo_landscape.svg
    footer: "Attendance Analysis"
execute: 
  echo: true
---




## Project Overview:

**Policy customers**: school accountability.

- Wanted a model to flag suspicious attendance data activity to identify, for example, whether a school had an unusual attendance pattern on an Ofsted visit day. 

**Data experts**: daily attendance data modelers.

- Description of the data model and its features such as retrospective data updates.

### Tools used:

- Databricks for data cleaning and data analysis
  - Allows the use of SQL and R scripts seamlessly in the same pipeline, as well as using Spark which optimises queries and distributes compute, efficiently processing the large dataset.
- Git for version control, repository hosted on Azure DevOps
  - Version control, particularly the use of branches in Git, allows for development and testing while keeping the main project safe. I used `git merge` when combining branches to retain the commit history of development branches.

## Starting off

- Understanding the daily attendance data model
- **Exploratory data analysis** 
  - Small number of schools with all pupil characteristics unknown e.g. SEN.
- Data cleaning - large dataset so investigative


There were duplicate entries for the same pupils. Discussions with data modellers revealed that some schools created new learner IDs when a pupil changed year groups, so we took the most recent learner ID.




``` {sql, eval=FALSE}
create or replace temporary view df_year_groups_deduped as
select a.*
from (
  select *, row_number() over (partition by source_learner_id order by admission_date_sk desc) as rn
  from df_year_groups
) as a
where a.rn = 1;
```




- Issues were communicated with the policy team so they could understand the data limitations early.

## Methodology: Entropy Analysis

Entropy measures the likelihood of certain attendance codes occurring on a given day, with **less likely** codes having **higher entropy**. 

<br>

Calculated as Shannon's entropy:

\begin{gather*}
H(X_{j,t}) = -\sum_{i = 0}^np(x_{i, j, t})\log_2p(x_{i, j, t})
\end{gather*}

Total entropy in school $j$ for session $t$ is equal to the sum for all **$n$ pupils** of the pupil's probability of having attendance code $x_{i,j,t}$ times by the log of that probability. 

How do we calculate the probability of each code?

- Code probabilities are modeled as a **Dirichlet distributed** random variable such that probabilities sum to 1 on any given session.

- Probabilities are estimated using **Bayesian updating** on a **pupil level**, so a pupil's most common code has the highest probability. 
  - The most recent attendance code has the highest weight because codes are persistent across sessions.
  - A pupil's attendance in the previous academic year provides the baseline probability

## Methodology: Entropy Analysis

:::: {.columns}
::: {.column width="69%"}
Drawbacks of this methodology:

  - Processing load: Intensive as we re-calculate a pupil's code probability each session -- with ~9 million pupils that's 18 million sessions per day!
    - Originally calculated probabilities using an R package `rdirchlet` but this calculation was done manually in spark SQL to allow for **efficient querying** using spark's distributed processing. 
    
    <br>
    
    - Created a function to run the model for specific schools rather than all schools, so we could test our outputs (shown on next slide). This meant we had a minimally viable product to show stakeholders the outputs, and explain the drawbacks. 
    
:::
::: {.column width=30%}
<br>
<br>
![](images/white_spark-removebg.png)
``` {.r code-line-numbers="1-5"}
plot_school_entropy(
  URN = "XXXXXXX",
  time_from = as.Date("2024-09-01"),
  time_to = as.Date("2024-11-24"),
  code_breakdown = TRUE)
```

:::
::::



## Entropy Methodology

::::{.columns}
:::{.column width=42%}
Example of an entropy output for one school broken down by specific attendance codes.

<br>

- Outputs need domain knowledge to be interpretable.
- Creates a significant number of false positives.
- The majority of entropy is coming from the most common attendance code, uninformative. 
- Assigned probabilities are crude, need a more sophisticated methodology like a neural network to more accurately predict attendance codes.

:::
:::{.column width=54%}
::::: {style="color: #000; background-color: #fff; padding: 10px; border: solid; border-color: #000; text-align: left;"}
![](images/m1_school_entropy_void.png)
:::::
:::
::::

## New Methodology: 

- Presented the limitations of the entropy model and explained the additional resource requirement to get more reliable results with less false positives.

<br> 

- Proposed an **alternative methodology** based on their initial requirements.
  - Looks at attendnace on **specific days** instead of every day for every school.
  - This means in their Ofsted example, we can look at attendance on the Ofsted inspection date.
  - The model is simplified to look at attendance and absence of specific pupil groups.
  
  
## Results of the adjusted methodology:

::::{.columns}
:::{.column width=51%}

The adjusted model flagged schools for further analysis after ofsted visits, and raised significantly less false positives. Benefits:
- Less computationally expensive
- More interpretable results
- Relevant to the current policy climate (this was during the time when Ofsted visits were being reformed)
- Code was flexible and reproducible so could be applied in other contexts e.g. looking at attendance on census days.

:::
:::{.column width=48%}
![](images/m2_school_ofsted_dip.jpg)
:::
::::

## Lessons learned

- Don't let efforts go to waste! 
  - The initial work was used as further evidence for investment in a neural network to predict pupil attendance codes. We currently have a contractor working on this and after 3-months the neural network has >80% accuracy!
  
- Noting users' requirements early is crucial, it made presenting a new methodology possible. 
  - Similarly, identifying a minimally viable product is important in case of future road blocks.

- It's okay for projects to pivot as long as the reasons are communicated early and effectively. Always come prepared with potential solutions!


::: center-content
## Any questions?

Slides built from the DfE analytical services' [Quarto template](https://github.com/dfe-analytical-services/quarto-slides-template)
:::